# Time Series Modelling
I can manually put the lagged variables in the features and then mask the ones for the k values of the model.
Will this work? Parameters are trained for all the values but we might need seperate parameters for each.

- It will not work. To train ARIMA, I need different parameter trineers for each model which is incredibly expensive computationally. We an try a linear approach which might be faster
- Global XGBoost might work
    - Boosting is an ensemble technique where new models are added to correct the errors made by existing models. Models are added sequentially until no further improvements can be made. 
    - Gradient boosting is an approach where new models are created that predict the residuals or errors of prior models and then added together to make the final prediction. It is called gradient boosting because it uses a gradient descent algorithm to minimize the loss when adding new models.

In [6]:
import pandas as pd
from pathlib import Path
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV


In [10]:
data = pd.read_parquet(Path("data") / "train.parquet")

In [12]:
def _merge_external_data(X):
    file_path = "external_data.csv"
    df_ext = pd.read_csv(file_path, parse_dates=["date"])

    X_comb = X.join(df_ext.set_index("date"), on="date") 
    X_comb.fillna(method="ffill", inplace=True)
    return X_comb.drop("Unnamed: 0", axis=1)

In [13]:
df = _merge_external_data(data)
df.head()

,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,coordinates,counter_technical_id,latitude,longitude,log_bike_count,Temperature (C),wind_dir,wind_speed,Humidity,Visibility,Precipitation,pressure1,holidays2
48321,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 02:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.000000,13.0,ONO,1.9,79,20.0,0.0,1019.9,0.0
48324,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,1.0,2020-09-01 03:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.693147,12.9,NO,1.9,81,20.0,0.0,1019.8,0.0
48327,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 04:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.000000,12.4,ONO,3.7,83,20.0,0.0,1019.5,0.0
48330,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,4.0,2020-09-01 15:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,1.609438,19.8,NNE,7.4,46,20.0,0.0,1017.5,0.0
48333,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,9.0,2020-09-01 18:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,2.302585,18.9,NE,5.6,46,20.0,0.0,1017.6,0.0


In [14]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols = list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
    # put it all together
    agg = concat(cols, axis=1)
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg.values